# How to use the StoG class from PyStoG

This tutorial shows how to use the `StoG` class from `pystog`


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import os
import numpy as np

from tests.materials import Argon
from tests.utils import get_index_of_function
from pystog.utils import ReciprocalSpaceHeaders, RealSpaceHeaders

from pystog import StoG

def cat(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        for line in lines:
            print(line.strip())

In [ ]:
import pystog
data_dir = os.path.join('..', 'tests', 'test_data')

## Preview of the input data using Argon

To use the `StoG` class, we first need some data. Lets load in some test data for Argon from our test suite. 
We are going to extract the $Q$ and $S(Q)$ along with the $r$ and $g(r)$. 

NOTE: The argon data was created using small molecular dynamics simualtions with [LAMMPS](https://lammps.sandia.gov/). The input files are provided in the PyStoG repository found [here](https://github.com/marshallmcdonnell/pystog/tree/master/data/lammps_inputs).

First, we can preview what is in the reciprocal space data file


In [ ]:
cat(os.path.join(data_dir, 'argon.reciprocal_space.dat'))

And also preview what is in the real space data file 

In [ ]:
cat(os.path.join(data_dir, 'argon.real_space.dat'))

## Initialize StoG input

Okay, so we create an instance of the `material` object using the test utility `Argon` class that helps us get the input ready for these datasets.

In [ ]:
material = Argon()

Now, we load in the reciprocal space data we previewed, specifically $S(Q)$:

In [ ]:
reciprocal_space_filename = os.path.join(data_dir, 'argon.reciprocal_space.dat')

data = np.loadtxt(reciprocal_space_filename, skiprows=2)
q = data[:, get_index_of_function("Q", ReciprocalSpaceHeaders)]
sq = data[:, get_index_of_function("S(Q)", RealSpaceHeaders)]

And also the real space data we previewed, specifically $g(r)$:

In [ ]:
real_space_filename = os.path.join(data_dir, 'argon.real_space.dat')
                 
data = np.loadtxt(real_space_filename, skiprows=2)
r = data[:, get_index_of_function("r", RealSpaceHeaders)]
gofr = data[:, get_index_of_function("g(r)", RealSpaceHeaders)]

Here, we use the material information along with the loaded $r$ data to make inputs we 
will pass to initialize the `StoG` class for this Argon data.

In [ ]:
kwargs_for_stog_input = {
    'NumberDensity': material.kwargs['rho'],
    '<b_coh>^2': material.kwargs['<b_coh>^2'],
    '<b_tot^2>': material.kwargs['<b_tot^2>'],
    'FourierFilter': {'Cutoff': 1.5},
    'OmittedXrangeCorrection': False,
    'Rdelta': r[1] -r[0],
    'Rmin': min(r),
    'Rmax': max(r)
}


And also the information to load in the reciprocal space data. Here we are showing how we 
can load two datasets in order to merge them together. One dataset will be for the 0->15.0 $Q$-space data
and the other will be from 1.9->35. Thus we will have overlap in the 1.9->15.0 $Q$-space range:

In [ ]:
kwargs_for_files = {
    'Files': [
        {'Filename': reciprocal_space_filename,
         'ReciprocalFunction': 'S(Q)',
         'Qmin': 0.02,
         'Qmax': 15.0,
         'Y': {'Offset': 0.0,
               'Scale': 1.0},
         'X': {'Offset': 0.0}
         },
        {'Filename': reciprocal_space_filename,
         'ReciprocalFunction': 'S(Q)',
         'Qmin': 1.90,
         'Qmax': 35.2,
         'Y': {'Offset': 0.0,
               'Scale': 1.0},
         'X': {'Offset': 0.0}
         }
    ]
}

## Using the StoG class with the input

Okay, now we initialize the `StoG` class with the information we passed:

In [ ]:
stog = StoG(**kwargs_for_stog_input)
stog.files = kwargs_for_files['Files']

Similarly, we could initialize the class with one input as below:

In [ ]:
kwargs_for_stog_input['Files'] = kwargs_for_files['Files']
stog = StoG(**kwargs_for_stog_input)

Using `StoG`, we can perform a workflow of reading in the data sets specified
in the `kwargs_for_files` information stored in the `stog.files` attribute, merge 
these datasets, transform them to reciprocal space, and then Fourier filter them with the 
cutoff we specified in `kwargs_for_stog_input`:

In [ ]:
stog.read_all_data()
stog.merge_data()
stog.transform_merged()
stog.fourier_filter()

Even though we have already processed the files, we can view the dianostics from the initial merge:

In [ ]:
stog.plot_merged_sq()

We can also view the processed $S(Q)$, the filter applied, and the resulting $Q[S(Q)-1]$

In [ ]:
stog.plot_sq()

And finally, we can see the resulting real space data for each step as well:

In [ ]:
stog.plot_gr()